In [1]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()
conn = psycopg2.connect(os.getenv("PGVECTOR_DB_URL"))

with conn:
    with conn.cursor() as cur:
        print("🔧 Adding 'filing_id' to exhibit_summaries...")

        cur.execute("""
            ALTER TABLE exhibit_summaries
            ADD COLUMN IF NOT EXISTS filing_id INTEGER;
        """)

        print("🔗 Linking existing exhibit_summaries to filing_summaries by accession...")
        cur.execute("""
            UPDATE exhibit_summaries es
            SET filing_id = fs.id
            FROM filing_summaries fs
            WHERE es.accession = fs.accession;
        """)

        print("🔒 Adding FOREIGN KEY constraint...")
        cur.execute("""
            ALTER TABLE exhibit_summaries
            ADD CONSTRAINT fk_filing
            FOREIGN KEY (filing_id) REFERENCES filing_summaries(id)
            ON DELETE CASCADE;
        """)

print("✅ Migration complete.")


🔧 Adding 'filing_id' to exhibit_summaries...
🔗 Linking existing exhibit_summaries to filing_summaries by accession...
🔒 Adding FOREIGN KEY constraint...
✅ Migration complete.


In [2]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv

load_dotenv()
conn = psycopg2.connect(os.getenv("PGVECTOR_DB_URL"))

query = """
    SELECT fs.ticker, fs.company, fs.filing_date, fs.accession,
           es.exhibit_label, LEFT(es.summary, 250)
    FROM filing_summaries fs
    LEFT JOIN exhibit_summaries es ON fs.id = es.filing_id
    ORDER BY fs.filing_date DESC, fs.accession;
"""

df = pd.read_sql(query, conn)
conn.close()

df.head(10)


C:\Users\Kris\AppData\Local\Temp\ipykernel_10428\2519406485.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ticker,company,filing_date,accession,exhibit_label,left
0,NVDA,NVIDIA CORP,2025-04-21,0001045810-25-000082-index.html,None,None
1,GPN,GLOBAL PAYMENTS INC,2025-04-21,0001104659-25-035771-index.html,exhibit 99.1,### Key Material Disclosures:\n\n- **Acquisiti...
2,GPN,GLOBAL PAYMENTS INC,2025-04-21,0001104659-25-035771-index.html,exhibit 99.2,### Key Material Disclosures:\n\n- **Transacti...
3,ELV,"Elevance Health, Inc.",2025-04-21,0001156039-25-000051-index.html,None,None
4,MRVL,"Marvell Technology, Inc.",2025-04-21,0001193125-25-079772-index.html,None,None
5,GOOG,Alphabet Inc.,2025-04-21,0001652044-25-000033-index.html,None,None
6,ASO,"Academy Sports & Outdoors, Inc.",2025-04-21,0001817358-25-000022-index.html,ex-99.1,### Key Material Disclosures:\n\n- **Financial...
7,TEST,Test Company,2025-04-20,0000000000-25-000001,None,None


In [4]:
import sys, os
sys.path.append(os.path.abspath(".."))

from agent_plugins.vector_store import connect
import pandas as pd

with connect() as conn:
    df = pd.read_sql("""
        SELECT
            id,
            ticker,
            company,
            filing_date,
            accession,
            exhibit_label,
            LEFT(summary, 300) as summary_preview
        FROM exhibit_summaries
        ORDER BY filing_date DESC
        LIMIT 10;
    """, conn)

display(df)


C:\Users\Kris\AppData\Local\Temp\ipykernel_10428\758538230.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


,id,ticker,company,filing_date,accession,exhibit_label,summary_preview
0,2,GPN,GLOBAL PAYMENTS INC,2025-04-21,0001104659-25-035771-index.html,exhibit 99.2,### Key Material Disclosures:\n\n- **Transacti...
1,1,GPN,GLOBAL PAYMENTS INC,2025-04-21,0001104659-25-035771-index.html,exhibit 99.1,### Key Material Disclosures:\n\n- **Acquisiti...
2,3,ASO,"Academy Sports & Outdoors, Inc.",2025-04-21,0001817358-25-000022-index.html,ex-99.1,### Key Material Disclosures:\n\n- **Financial...


In [5]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()
conn = psycopg2.connect(os.getenv("PGVECTOR_DB_URL"))

with conn:
    with conn.cursor() as cur:
        print("🧬 Adding raw_text to filing_summaries (if missing)...")
        cur.execute("""
            ALTER TABLE filing_summaries
            ADD COLUMN IF NOT EXISTS raw_text TEXT;
        """)

        print("🧬 Adding exhibit_text to exhibit_summaries (if missing)...")
        cur.execute("""
            ALTER TABLE exhibit_summaries
            ADD COLUMN IF NOT EXISTS exhibit_text TEXT;
        """)

print("✅ Columns ensured.")


🧬 Adding raw_text to filing_summaries (if missing)...
🧬 Adding exhibit_text to exhibit_summaries (if missing)...
✅ Columns ensured.
